In [124]:
import numpy as np
import pandas as pd

In [255]:
sheet_names = ['A1985', 'A1986', 'A1987', 'A1988', 'A1989', 'A1990', 'w']
years = [str(1985 + i) for i in range(0, 6)]
data = pd.read_excel("full_table.xlsx", sheet_name = sheet_names)
table = {}
for name in sheet_names:
    table[name] = data[name].values

for i in range(1985, 1991):
    s = 'w' + str(i)
    table[s] = table['w'][:, i - 1985]

In [256]:
def is_productive(A, w): # returns [x, Ax, Ax + w, is_product]
    x = np.linalg.inv(np.eye(A.shape[0]) - A) @ w
    return [x, A @ x, A @ x + w, np.all(x >= 0)]

In [257]:
def perron_frobenius(A): # returns [lam, xA] - Perron-Frobenius eigenvalue and eigenvector
    r = np.random.rand(A.shape[0])
    r_old = r / np.linalg.norm(r)
    dif = 1
    it = 0
    while np.linalg.norm(dif) > 0.00001:
        it += 1
        r = A @ r / (np.linalg.norm(A @ r))
        dif = r - r_old
        r_old = r
        lam = r @ A @ r
    #print('Algorithm converged in', it, 'iterations.')
    return [lam, r]

In [354]:
def delete_sectors(A, to_del):
    to_del_c = list({i for i in range(0, A.shape[0])}.difference(set(to_del)))
    A22 = A[:, to_del][to_del]
    A11 = A[:, to_del_c][to_del_c]
    A12 = A[:, to_del][to_del_c]
    A21 = A[:, to_del_c][to_del]
    B = A11 + A12 @ np.linalg.inv(np.eye(A22.shape[0]) - A22) @ A21
    return B

In [387]:
def aggregate(A, w, to_aggr):
    n = to_aggr.shape[0]
    B = np.zeros((n, n))
    x = np.linalg.inv(np.eye(A.shape[0]) - A) @ w
    for i in range(0, n):
        for j in range(0, n):
            B[i][j] = (A[:, to_aggr[j]][to_aggr[i]] @ x[to_aggr[j]]).sum() / (x[to_aggr[j]].sum())
    return B

In [350]:
A = np.array([[2, 2, 2], [3, 4, 5], [6, 7, 8]])
delete_sectors(A, [1])


[1] [0, 2]


array([[ 0.   , -1.333],
       [-1.   , -3.667]])

In [349]:
A

array([[2, 2, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [258]:
df = pd.DataFrame({
    'Productive': np.zeros(6),
    'PF_eigenvalue': np.zeros(6)},
    index=years)

In [259]:
for year in years:
    res = is_productive(table['A' + year], table['w' + year])
    fp = perron_frobenius(table['A' + year])
    table['x' + year] = res[0]
    table['fp' + year] = fp[1]
    df.loc[year, 'Productive'] = res[3]
    df.loc[year, 'PF_eigenvalue'] = fp[0]
df

,Productive,PF_eigenvalue
1985,True,0.550485
1986,True,0.551078
1987,True,0.543420
1988,True,0.536549
1989,True,0.533277
1990,True,0.538223


In [275]:
X = np.zeros((18, 6))
X_fp = np.zeros((18, 6))
for year in years:
    X[:, int(year) - 1985] = table['x' + year]
    X_fp[:, int(year) - 1985] = table['fp' + year]

%precision 2
print('X = \n', X)

%precision 4
print('\nPerron-Frobenius vectors:\n', X_fp)

X = 
 [[ 58129.03  59896.06  62723.26  63517.06  63916.66  67472.43]
 [142528.65 144400.97 145490.58 147039.85 145662.51 149868.89]
 [ 33282.24  34243.59  31247.45  31854.41  31118.2   31574.13]
 [  1091.06   1211.91   1115.92   1180.93   1144.07   1098.26]
 [102392.92 105769.88 105855.29 107269.87 106469.32 104340.32]
 [ 83419.63  85632.87  88206.07  89590.39  94240.2  113408.56]
 [118115.11 125119.46 123221.01 129845.99 133362.64 134430.58]
 [303592.63 318757.33 334312.83 344057.   351097.15 341339.62]
 [ 72369.16  75461.1   74718.69  76037.48  78668.9   79684.15]
 [ 61198.58  64884.77  63911.74  65058.29  67300.15  66347.39]
 [190058.51 192665.11 185663.84 189717.84 194848.9  196675.88]
 [157348.4  156674.27 162540.21 168299.32 179999.66 192151.53]
 [ 42361.05  44488.3   46534.11  48676.68  52596.84  59584.77]
 [ 82269.    91224.    96327.   103787.   107206.   108425.  ]
 [263487.54 280862.6  289167.64 333268.35 344325.54 373789.12]
 [144041.79 150413.47 147826.38 156982.57 149615.

In [433]:
%precision 3
A = table['A1985']
to_del = [12, 13, 17]
to_del_c = list({i for i in range(0, A.shape[0])}.difference(set(to_del)))

A_del = delete_sectors(A, to_del)
A_del.shape

(15, 15)

In [436]:
# sector deletion
w = table['w1985']
w[to_del] = 0
x = np.linalg.inv(np.eye(A.shape[0]) - A) @ w
x_del = x[to_del_c]
w_del = w[to_del_c]

# check after deletion
x_del - A_del @ x_del - w_del

array([-1.819e-12,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
        9.095e-13, -3.638e-12,  3.638e-12,  3.638e-12,  0.000e+00,
        0.000e+00,  0.000e+00,  5.821e-11, -7.276e-12,  1.455e-11])

In [437]:
# aggregation
to_aggr = np.array([[0, 1, 2, 3], [4, 5, 6, 7], [8, 9, 10, 11, 12, 13, 14]])
A_aggr = aggregate(A_del, w_del, to_aggr)
A_aggr

array([[0.294, 0.076, 0.032],
       [0.027, 0.427, 0.05 ],
       [0.168, 0.116, 0.468]])

In [438]:
# aggregation check
w_new = np.array([w_del[to_aggr[i]].sum() for i in range(0, to_aggr.shape[0])])
x_new = np.linalg.inv(np.eye(A_aggr.shape[0]) - A_aggr) @ w_new
x_new_sum = np.array([x_del[to_aggr[i]].sum() for i in range(0, to_aggr.shape[0])])
x_new - x_new_sum

array([-7.276e-12, -7.276e-12, -1.164e-10])